# Школа алготрейдеров. Блок торгового ПО и программирования
## Занятие 8. Использование системы Quanteon

### Стратегия 1. Buy & Hold (Купи и держи)

#### Краткое описание
Купить актив и держать его всё время работы стратегии.

#### Подробное описание
1. Заполните базовые настройки стратегии. В данном случае рекомендуется использовать дневки, набор торговых инструментов — любой, позиции — относительные.
2. Торговая функция должна возвращать веса, равные 1, для всех торговых инструментов.

#### Шаблон

In [ ]:
# -*- coding: utf-8 -*-


def mySettings():
    settings = {}

    settings['market'] = 'FORTS1D'
    settings['dataRequest'] = {'trade_assets': [...]}
    settings['capital'] = 1000000
    settings['pos_type'] = 'relative'

    settings['lookback'] = 252

    return settings


def myTradingSystem(data, settings):

    ...

    return pos, settings


if __name__ == '__main__':
    import quanteonToolbox as qtb
    backTester = qtb.BackTester(__file__)
    result = backTester.run()
    print result['stats']['general']
    backTester.plot()

#### Подсказка
Чтобы получить массив единичных весов некоторого размера `n`, можно воспользоваться одним из следующих приёмов:

In [ ]:
p = [1] * n
p = np.ones(n)

### Стратегия 2. 2-SMA trend following

#### Краткое описание
В каждый момент времени насчитывать 2 индикатора — длинное и короткое скользящее среднее. Размеры окон для этих скользящих средних являются параметрами стратегии. Держать актив в лонге, если короткая скользящая средняя больше длинной скользящей средней, в противном случае держать актив в шорте.

#### Подробное описание
1. Заполните базовые настройки стратегии. Рекомендуется использовать дневки и инструмет Si, позиции — относительные.
1. Добавьте настройки `period_long` и `period_short`. Выберите их равными, например, 22 и 7 соответственно.
1. В торговой функции необходимо вычислить 2 скользящих средних.
1. Если короткая средняя больше длинной, вернуть вес 1, в противном случае вернуть вес −1.
1. (*) Можете попробовать реализовать в этой стратегии набор инструментов вместо одного. Пакет NumPy позволяет вычислять всё в векторном виде, без использования циклов for, поэтому количество кода не должно существенно измениться.

#### Подсказка
Для вычисления, например, длинной скользящей средней можно воспользоваться следующим кодом.

In [ ]:
sma_long = sum(data['CLOSE'][-settings['period_long']:, 0]) / settings['period_long']

В случае нескольких активов поможет использование функции `numpy.mean`.

### Стратегия 3. RTS breakout long

#### Краткое описание
Простейшая пробойная стратегия. Она торгует внутри дня фьючерсом на индекс РТС по следующему принципу.

В начале каждого дня считается уровень сопротивления как некая функция от `OPEN`, `HIGH`, `LOW`, `CLOSE` предыдущего дня.
В дальнейшем в случае, если цена проходит уровень сопротивления, заходим в лонг в ожидании, что в этот день происходит рост рынка и цена пойдет дальше вверх. Выходим из сделки в 23:45 или по стоп-лоссу.

#### Подробное описание
1. Заполните базовые настройки стратегии. Используем пятиминутки, инструмент RTS, позиции абсолютные, торговать всегда будем одним лотом. Используем `lookback`, равный количеству пятиминуток в торговую сессию плюс один, то есть 167.
1. `OPEN`, `HIGH`, `LOW`, `CLOSE` предыдущего дня, уровень сопротивления и уровень стоп-лосса также будут настройками. Они будут насчитываться в процессе работы стратегии, поэтому создайте соответствующие поля в словаре настроек, но не присваивайте им никакого значения.
1. Также для удобства рекомендуется хранить в настройках текущую позицию (и соответственно не забывать обновлять её в процессе работы стратегии).
1. В случае, если сейчас время 10:00, рассчитайте `OPEN`, `HIGH`, `LOW`, `CLOSE` предыдущего дня. Рассчитайте уровень сопротивления по формуле `0.5*O + 0.5*H - 0.5*L + 0.5*C`.
1. В случае, если время сейчас не меньше чем 23:45, возвращем нулевую позицию.
1. Если сейчас позиция равна нулю, проверяем, что цена закрытия превысила уровень сопротивления. Если это так, устанавливаем уровень стоп-лосса как 98,5 % от уровня закрытия и возвращаем позицию, равную единице.
1. Если сейчас позиция равна нулю, проверяем, опустилась ли цена закрытия ниже уровня стоп-лосса. Если это так, закрываем позицию, то есть возвращаем нулевое значение.
1. Если изменения позиции не произошло, возвращаем ту же позицию, которая была.

#### Шаблон

In [ ]:
# -*- coding: utf-8 -*-


def mySettings():
    settings = {}

    settings['market'] = 'FORTS5M'
    settings['dataRequest'] = {'trade_assets': ['RTS']}
    settings['capital'] = 200000

    # day (166 5-minutes) and one more minute
    settings['lookback'] = 166 + 1

    ...

    return settings


def myTradingSystem(data, settings):
    ...


if __name__ == '__main__':
    import quanteonToolbox as qtb
    backTester = qtb.BackTester(__file__)
    result = backTester.run()
    print result['stats']['general']
    backTester.plot()

#### Подсказки
1. Чтобы проверить, является ли текущий бар началом дня, можно привести последний timestamp к строке и обрезать её по часам и минутам: `cur_hour_minute = str(data['TIMESTAMP'][-1])[-6:-2]`
2. Чтобы насчитать `OPEN`, `HIGH`, `LOW`, `CLOSE` предыдущего дня, пользуемся тем, что `lookback = 167`. Например, `HIGH` предыдущего дня считается по формуле `H = max(data['HIGH'][:-1,0])`
3. Установить стоп-лосс можно следующей строчкой: `s['stoplevel'] = C * 0.985`